In [6]:
from linkingtool import RESources as RES

In [7]:
from linkingtool.hdf5_handler import DataHandler
import linkingtool.visuals as vis
store=f"data/store/resources_AB.h5"
datahandler=DataHandler(store)

# Set Required Args to Activate Modules

In [8]:
# Iterate over provinces for both solar and wind resources
resource_types = ['wind','solar'] # , 
provinces=['BC']  #'AB','SK','ON','NS'
for province_code in provinces:
    for resource_type in resource_types:
        required_args = {
            "config_file_path": 'config/config.yaml',
            "province_short_code": province_code,
            "resource_type": resource_type
        }
        
        # Create an instance of Resources and execute the module
        res_module = RES.RESources_builder(**required_args)

In [9]:
cells=datahandler.from_store('cells')

[Assumptions](https://www.bchydro.com/content/dam/BCHydro/customer-portal/documents/corporate/regulatory-planning-documents/integrated-resource-plans/current-plan/rou-characterization-wind-report-20150519-hatch.pdf)
The following assumptions were considered as common to all regions: 
• 3MW turbine model  
• On-shore projects (land-based)
• Project  lifetime of 20 years  
• Crown Land only  
• Costs in 2024$ (CAD) assuming delivery in 2025 
• Project development timelines can vary and the costs for pre construction of the  template project will be based on 5 years of pre development work until reception of  electricity purchase agreement (EPA) and beginning of construction 
• When considering the developer’s internal costs, Hatch has assumed a development 
model where all pre-development costs are absorbed by the same stakeholder that is 
involved in the later stages of the project, including development, procurement, 
construction, BOP, O&M, etc.   This model was chosen over a model where, for 
example, a project has undergone some level of pre-development and then the 
project rights and assets are sold to a new stakeholder prior to construction or 
operation.  Such transaction costs cannot be easily or simply quantified and so Hatch 
considered a simplified model for the purposes of this mandate. Costs associated 
with procuring assets from another developer are not evaluated.   

• The developer is assumed to be experienced, working in a competitive market and to 
be medium or average in size (i.e. not a public utility and not a private, small scale 
independent developer with limited experience)

In [11]:
def calc_LCOE_lambda(self,
                        row):
    
    """ 
    # Method: 
    LCOE = (FCR * TCC + FOC + GCC + TRC) / AEP + VOC)
        - Total Capital cost, $ (TCC)
        - Fixed annual operating cost, $ (FOC)
        - Variable operating cost, $/kWh (VOC)
        - Fixed charge rate (FCR)
        - Annual electricity production, kWh (AEP)
        - Grid Connection Cost (GCC)
        - Transmission Line Rebuild Cost (TRC) 
        
    ### Ref: 
    - https://sam.nrel.gov/financial-models/lcoe-calculator.html
    - https://www.nrel.gov/docs/legosti/old/5173.pdf
    - https://www.nrel.gov/docs/fy07osti/40566.pdf
    
    """

    dtg = row['nearest_station_distance_km'] # km
    gcc_pu = row[f'grid_connection_cost_per_km_{self.resource_type}'] # m$/km
    gcc=dtg*gcc_pu/1.60934  # Convert to miles as our costs are given in m$/miles (USA study)
    trc=row[f'tx_line_rebuild_cost_{self.resource_type}']/ 1.60934 # m$/km
    capex = row[f'capex_{self.resource_type}'] # m$/km
    foc = row[f'fom_{self.resource_type}'] * row[f'potential_capacity_{self.resource_type}'] # $/ MW * MW
    voc = row[f'vom_{self.resource_type}'] * row[f'potential_capacity_{self.resource_type}'] # $/ MW * MW
    fcr = row.get('FCR', 0.098) 
    aep = 8760 * row[f'{self.resource_type}_CF_mean'] * row[f'potential_capacity_{self.resource_type}'] # MWh
    tcc= capex + fcr * tcc
    I_0=
    
    
    if aep == 0: # some cells have no potentials
        return float(99999)  # handle the error 
    else:
        lcoe = ((fcr * tcc + gcc + trc + foc) / aep + voc)  # m$/MWh
        return lcoe  * 1E6 # LCOE in $/MWh      

In [ ]:
dataframe[f'LCOE_{self.resource_type}'] = dataframe.apply(lambda row: self.calc_LCOE_lambda(row), axis=1) # adopting NREL's method + some added costs
            
            # scored_dataframe = dataframe.sort_values(by=f'lcoe_{self.resource_type}', ascending=False).copy()  # Lower LCOE is better
            scored_dataframe = dataframe.sort_values(by=f'LCOE_{self.resource_type}', ascending=False).copy()  # Lower LCOE is better
            
            return scored_dataframe

In [10]:
cells.head(2)

,x,y,Country,Province,Region,geometry,potential_capacity_solar,capex_solar,fom_solar,vom_solar,...,windspeed_ERA5,windspeed_gwa,CF_IEC2,CF_IEC3,x_2,y_2,nearest_station,nearest_station_distance_km,lcoe_wind,Operational_life_wind
cell,,,,,,,,,,,,,,,,,,,,,
DivisionNo.3_-114.25_49.0,-114.25,49.0,Canada,Alberta,DivisionNo.3,"POLYGON ((-114.12500 49.12500, -114.12500 49.0...",0.0,0,0,0,...,NaN,7.341490,0.365528,0.393708,-114.25,49.0,AB_502S_ISS,24.095365,22.597503,20
DivisionNo.3_-114.0_49.0,-114.00,49.0,Canada,Alberta,DivisionNo.3,"POLYGON ((-114.12500 49.12500, -113.87500 49.1...",0.0,0,0,0,...,NaN,9.724767,0.461598,0.486244,-114.00,49.0,AB_502S_ISS,20.298534,12.859958,20
